In [3]:
import cv2
import os

# Define the video file path
video_path = 'clip.mp4'

# Create a directory to save the frames
frames_dir = 'frames'
os.makedirs(frames_dir, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(video_path)
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Save frame as image
    frame_path = os.path.join(frames_dir, f'frame_{frame_count:04d}.jpg')
    cv2.imwrite(frame_path, frame)
    
    frame_count += 1

cap.release()
print(f'Extracted {frame_count} frames to {frames_dir}')


Extracted 867 frames to frames


In [4]:
import torch
import cv2
import os
from pathlib import Path

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Define the path to the frames directory
frames_dir = Path('frames')

# Define the output directory for annotated frames
output_annotated_dir = Path('annotated_frames')
output_annotated_dir.mkdir(exist_ok=True)

# Perform object detection on each frame
for frame_file in frames_dir.iterdir():
    if frame_file.suffix == '.jpg':
        # Read the frame
        img = cv2.imread(str(frame_file))
        
        # Perform detection
        results = model(img)
        
        # Parse results
        for result in results.xyxy[0].cpu().numpy():
            x1, y1, x2, y2, conf, cls = result
            label = model.names[int(cls)]
            if label == 'syringe':  # Assuming 'syringe' is the label for the syringe class
                # Draw bounding box
                cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(img, f'{label} {conf:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Save the annotated frame
        annotated_frame_path = output_annotated_dir / frame_file.name
        cv2.imwrite(str(annotated_frame_path), img)

print(f'Annotated frames saved in {output_annotated_dir}')


Using cache found in C:\Users\Lenovo/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-17 Python-3.10.7 torch-2.3.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Annotated frames saved in annotated_frames


In [5]:
output_video_path = 'annotated_clip.mp4'
frame_files = sorted([f for f in output_annotated_dir.iterdir() if f.suffix == '.jpg'])
frame = cv2.imread(str(frame_files[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))

for frame_file in frame_files:
    frame = cv2.imread(str(frame_file))
    video.write(frame)

video.release()
print(f'Annotated video saved at {output_video_path}')


Annotated video saved at annotated_clip.mp4
